# Semi-Automated Plate Definition Generation

- tags: #platedefinition #resourcemovement #plateadapter #hamiltonstar
- Last updated: 2026-01-26

## Prerequisites

- Machines used:
  - Hamilton STAR
- Non-PLR dependencies: None 


## Preview of Machine Behvaiour

<video width="640" controls autoplay loop>
  <source src="./assets/star_movement_plate_to_alpaqua_core/animation.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

## Protocol Mode

In [1]:
protocol_mode = "simulation" # "execution" or "simulation"

---
## Import Statements

### Non-PLR Dependencies

None

### Machine & Visualizer

In [2]:
%load_ext autoreload
%autoreload 2
       
import random 
import time

from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.resources.hamilton import STARLetDeck
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.utils import chunk_list

if protocol_mode == "execution":

  from pylabrobot.liquid_handling.backends import STARBackend

  star = STARBackend()

elif protocol_mode == "simulation":

  from pylabrobot.liquid_handling.backends.hamilton.STAR_chatterbox import STARChatterboxBackend
    
  star = STARChatterboxBackend()

### Required Resources

In [3]:
from pylabrobot.resources import (
    hamilton_mfx_carrier_L5_base,
    hamilton_mfx_plateholder_DWP_metal_tapped,
    hamilton_mfx_plateholder_DWP_flat,
    Coordinate
)

#### Create Initial Plate Definition

In [4]:
from pylabrobot.resources.plate import Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
    CrossSectionType,
    Well,
    WellBottomType,
)

def biorad_96_wellplate_200uL_Vb(name: str, with_lid: bool = False) -> Plate:
    """Bio-rad cat. no.: HSP9601 (50/package), HSP9601B (8*50/package).

    Bio-rad plate with 96-wells (V-bottoms).
    "The patented rigid 2-component design is specifically engineered to
      withstand the stresses of thermal cycling." -> excellent for automation!

    - Colour: white shell/clear well
    - alternative cat. no.:
      - red shell (HSP-9611)
      - yellow shell (HSP-9621)
      - blue shell (HSP-9631)
      - green shell (HSP-9641)
      - black shell (HSP-9661)
    - Material: Polypropylene
    - Cleanliness: "Certified to be free of DNase, RNase, and human genomic DNA"
    - Total volume/well = 200 uL
    - URL: https://www.bio-rad.com/en-uk/sku/HSP9601-hard-shell-96-well-pcr-plates-
      low-profile-thin-wall-skirted-white-clear?ID=HSP9601
    - technical drawing: ./techical_drawings/biorad_96_wellplate_200uL_Vb.png
    """
    well_diameter = 5.46  # mm
    return Plate(
        name=name,
        size_x=127.76,
        size_y=85.48,
        size_z=16.06,
        lid=None,
        model=biorad_96_wellplate_200uL_Vb.__name__,
        ordered_items=create_ordered_items_2d(
            Well,
            num_items_x=12,
            num_items_y=8,
            dx=11.65,
            dy=8.51,
            dz=1.1,
            item_dx=9.0,
            item_dy=9.0,
            size_x=well_diameter,
            size_y=well_diameter,
            size_z=14.4,
            bottom_type=WellBottomType.V,
            material_z_thickness=0.65,
            cross_section_type=CrossSectionType.CIRCLE,
            compute_volume_from_height=None,
            compute_height_from_volume=None,
        ),
    )

plate_definition_function = biorad_96_wellplate_200uL_Vb

## Instantiate Frontend & Connect to Machine

In [5]:
deck = STARLetDeck()
lh = LiquidHandler(backend=star, deck=deck)

await lh.setup()

vis = Visualizer(resource=lh)
await vis.setup()

await star.disable_cover_control() # 😈

Websocket server started at http://127.0.0.1:2123
File server started at http://127.0.0.1:1339 . Open this URL in your browser.
C0CDid0001


## Configure Deck Layout

In [6]:
# Setup MFX Carrier with DWP PlateHolder that has a pedestal

plateholder_pedestal_0 = hamilton_mfx_plateholder_DWP_metal_tapped(
    name=f"plateholder_pedestal_0"
)

mfx_carrier_0 = hamilton_mfx_carrier_L5_base(
  name="mfx_carrier_0",
  modules={0: plateholder_pedestal_0}
)

deck.assign_child_resource(mfx_carrier_0, rails=1)

# Setup MFX Carrier with DWP PlateHolder that has a flat bottom

plateholder_flat_0 = hamilton_mfx_plateholder_DWP_flat(name=f"plateholder_flat_0")

mfx_carrier_1 = hamilton_mfx_carrier_L5_base(
  name="mfx_carrier_1",
  modules={0: plateholder_flat_0}
)

pcr_plate_0 = plate_definition_function(name="pcr_plate_0")

mfx_carrier_1[0] = pcr_plate_0 

deck.assign_child_resource(mfx_carrier_1, rails=8)

---
## Execution

### Store Coordinates to Probe on Flat

In [7]:
def generate_channel_wells(plate, channels_used):
    """
    Generate a list of wells for multi-channel pipetting with evenly spaced rows.
    
    Returns:
        List of well names (e.g., ["A1", "C1", "E1", "H1", "A2", "C2", ...])
    """
    num_rows, num_cols = plate.num_items_y, plate.num_items_x
    
    # Generate evenly spaced row indices
    row_indices = [round(i * (num_rows - 1) / (channels_used - 1)) for i in range(channels_used)]
    
    # Generate well names: all columns, with selected rows per column
    return [f"{chr(ord('A') + row)}{col}" 
            for col in range(1, num_cols + 1) 
            for row in row_indices]

In [8]:
n_channels = 3
wells_to_probe = generate_channel_wells(pcr_plate_0, channels_used=n_channels)

positions_to_probe_flat = [
    well.get_absolute_location("c", "c", "top")
    for well in pcr_plate_0.children
        if well.get_identifier() in wells_to_probe
]

### Move Plate to PlateHolder with Pedestal

In [9]:
teaching_tip_rack = lh.deck.get_resource("teaching_tip_rack")
await lh.pick_up_tips(teaching_tip_rack["A1:C1"], use_channels=[0,1,2,3,4,5][:n_channels])

await star.pick_up_core_gripper_tools(front_channel=7)

C0TTid0002tt01tf1tl0519tv03600tg2tu0
C0TPid0003xp07854 07854 07854 00000&yp5286 5196 5106 0000&tm1 1 1 0&tt01tp1830tz1750th2450td0
C0ZTid0004xs07975xd0ya1250yb1070pa07pb08tp2350tz2250th2800tt14


In [10]:
move_target = mfx_carrier_0[0]

if protocol_mode == "simulation":
  time.sleep(2)
    
await lh.move_plate(
  plate=pcr_plate_0,
  to=move_target,
  use_arm="core",
  pickup_distance_from_top=6,
  core_grip_strength=40,
  return_core_gripper=False,
)

if protocol_mode == "execution":
  # "smart" command, will ask operator for input if it cannot find plate in
  # move_target location place into condition for simulation mode

  # (1) check transfer success, (2) push plate flush
  await star.core_check_resource_exists_at_location_center(
    location=pcr_plate_0.get_absolute_location(),
    resource=pcr_plate_0,
    gripper_y_margin=9,
    enable_recovery=True,
    audio_feedback=False,
  )

print(star.core_parked)

if protocol_mode == "simulation":
  time.sleep(2)

C0ZPid0005xs03254xd0yj1142yv0050zj1881zy0500yo0885yg0825yw40th2800te2800
C0ZRid0006xs01679xd0yj1147zj1929zi000zy0500yo0885th2800te2800
False


### Store Coordinates to Probe on Pedestal

In [11]:
positions_to_probe_pedestal = [
    well.get_absolute_location("c", "c", "top")
    for well in pcr_plate_0.children
        if well.get_identifier() in wells_to_probe
]

### Probe True PlateHolder Origin

In [12]:
plate_holder_flat_child_coord = mfx_carrier_1[0].get_absolute_location()+mfx_carrier_1[0].child_location

test_location_w_offset = plate_holder_flat_child_coord + Coordinate(x=3, y=3, z=0)

In [17]:
channel_idx = 0

# Prepare Z position of X-Y probing
await star.move_all_channels_in_z_safety()
await star.prepare_for_manual_channel_operation(channel=channel_idx)

await star.move_channel_x(x=test_location_w_offset.x, channel=channel_idx)
await star.move_channel_y(y= test_location_w_offset.y, channel=channel_idx)

probed_z_position = await star.clld_probe_z_height_using_channel(
        channel_idx=channel_idx,
        channel_speed=6.0, start_pos_search=test_location_w_offset.z+20,
        detection_edge=5, post_detection_dist=5.0,
        move_channels_to_safe_pos_after=False
    ) if protocol_mode == "execution" else plate_holder_flat_child_coord.z
await star.move_channel_z(z=probed_z_position+1.5, channel=channel_idx)

# X-Y Probing
offset_n_replicates = 3

front_pos_list = []
for x in range(offset_n_replicates):
    probed_front_position = await star.clld_probe_y_position_using_channel(
        channel_idx=channel_idx,
        probing_direction="forward",
        channel_speed=3,
        post_detection_dist=3.0,
        end_pos_search=test_location_w_offset.y-5
    ) if protocol_mode == "execution" else plate_holder_flat_child_coord.y
    front_pos_list.append(probed_front_position)

left_pos_list =[]
for x in range(offset_n_replicates):
    probed_left_position = await star.clld_probe_x_position_using_channel(
        channel_idx=channel_idx,
        probing_direction="left",
        post_detection_dist=1.0,
        end_pos_search=test_location_w_offset.x-5
    ) if protocol_mode == "execution" else plate_holder_flat_child_coord.x
    left_pos_list.append(probed_left_position)

await star.move_all_channels_in_z_safety()

true_plate_holder_flat_child_coord = Coordinate(
    x=round(sum(left_pos_list)/len(left_pos_list), 1),
    y=round(sum(front_pos_list)/len(front_pos_list), 1),
    z=probed_z_position,
)

C0ZAid0023
C0JPid0024pn01
C0JXid0025xs02645
moving channel 0 to y: 74.5
C0KZid0026pn01zj1795
C0ZAid0027


### Move Plate back onto tapped PlateHolder

In [18]:
move_target = mfx_carrier_1[0]

await lh.move_plate(
  plate=pcr_plate_0,
  to=move_target,
  use_arm="core",
  pickup_distance_from_top=6,
  core_grip_strength=40,
  return_core_gripper=False,
)

if protocol_mode == "execution":

  await star.core_check_resource_exists_at_location_center(
    location=pcr_plate_0.get_absolute_location(),
    resource=pcr_plate_0,
    gripper_y_margin=9,
    enable_recovery=True,
    audio_feedback=False,
  )

C0ZPid0028xs01679xd0yj1147yv0050zj1929zy0500yo0885yg0825yw40th2800te2800
C0ZRid0029xs03254xd0yj1142zj1881zi000zy0500yo0885th2800te2800


### Interactive X-Y Verification of Bottom-Left Well

In [43]:
test_location = pcr_plate_0.column(0)[-1].get_absolute_location(
    x="center",y="center",z="top"
)

await lh.backend.move_all_channels_in_z_safety()

await lh.backend.prepare_for_manual_channel_operation(channel=channel_idx)

C0ZAid0130
C0JPid0131pn01


In [60]:
# Position channel with teaching needle above bottom-left well
test_x = test_location.x + 0.0

await lh.backend.move_channel_x(x=test_x, channel=channel_idx)

current_dx = round(test_x - true_plate_holder_flat_child_coord.x - pcr_plate_0.children[0].get_size_x()/2, 1)
print(f"{test_x}, {current_dx=}")

C0JXid0141xs02759
275.88, current_dx=11.6


In [61]:
test_y = test_location.y + 0.0

await lh.backend.move_channel_y(y=test_y, channel=channel_idx)

current_dy = round(test_y - true_plate_holder_flat_child_coord.y - pcr_plate_0.children[0].get_size_y()/2, 1)
print(f"{test_y}, {current_dy=}")

moving channel 0 to y: 82.74
82.74, current_dy=8.5


In [51]:
# Only to move the teaching needle into the well for X/Y correction
test_z = test_location.z + 0.0
await lh.backend.move_channel_z(z=test_z, channel=channel_idx)
test_z

C0KZid0136pn01zj1935


193.5

In [64]:
final_query = input(f"Have you updated the dx and dy?")
await lh.backend.move_all_channels_in_z_safety()


Have you updated the dx and dy? 


### Z Probing of Wells (on Flat PlateHolder)

In [68]:
if protocol_mode == "execution":

    probing_start_time = time.time()

    await lh.backend.move_all_channels_in_z_safety()
    use_channels = list(range(n_channels))
    await lh.backend.prepare_for_manual_channel_operation(channel=use_channels[-1])
    
    plate_on_flat_results = []
    
    for positions in chunk_list(positions_to_probe_flat, chunk_size=n_channels):
        
        x_pos = [coord.x for coord in positions]
        y_pos = [coord.y for coord in positions]
        z_start_pos = positions[0].z+20 # TODO: add size_z measurement
        
        
        await star.move_channel_x(0, x_pos[0])

        await star.position_channels_in_y_direction(
          {channel: y for channel, y in zip(use_channels, y_pos)}
        )
        await star.position_channels_in_z_direction(
          {channel: z_start_pos for channel in use_channels}
        )

        # Detect well cavity bottoms
        start_pos_search_list = [z_start_pos for x in range(n_channels)]
        measured_well_bottoms_replicate_summary = []
        tech_n_replicates = 3 

        for n in range(tech_n_replicates):

            # Parallelise ztouch probing
            # taking care to avoid parameters which require the unparallelizable C0 command module
            measured_well_bottoms = await asyncio.gather(
                  *[
                      star.ztouch_probe_z_height_using_channel(
                          channel_idx=channel_idx,
                          tip_len=lh.head[channel_idx].get_tip().total_tip_length,  # must be declared to avoid STAR C0 module
                          channel_speed=10.0,
                          start_pos_search=start_pos_z,
                          lowest_immers_pos=probed_z_position-2,
                          detection_limiter_in_PWM=0,
                          push_down_force_in_PWM=0,
                          post_detection_dist=0.0, # must be 0 to avoid STAR C0 module
                          move_channels_to_safe_pos_after=False  # must be False to avoid STAR C0 module
                    )
                    for channel_idx, start_pos_z in zip(
                        use_channels, start_pos_search_list
                    )
                  ]
                )
            measured_well_bottoms_replicate_summary.append(measured_well_bottoms)

            start_pos_search_list = [x+3 for x in measured_well_bottoms] # Accelerate

        mean_measured_well_bottoms = [
            sum(replicates)/tech_n_replicates
            for replicates in list(zip(*measured_well_bottoms_replicate_summary))
        ]
        plate_on_flat_results.append(mean_measured_well_bottoms)
        
        # SAFETY: move UP before starting next loop!
        await star.position_channels_in_z_direction(
          {channel: z_start_pos for channel in use_channels}
        )
    
    print(f"Well probing took {time.time() - probing_start_time} sec")
    

### Z Probing of PlateHolder w/ Pedestal

### Move Plate to PlateHolder with Pedestal

### Z Probing of Flat PlateHolder

### Z Probing of Wells (on Pedestal)

### Update Plate Definition